In [22]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout, Conv1D, MaxPooling1D, GRU


import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import os, re, json, functools


#dark mode
plt.rc_context({'xtick.color':'w', 'ytick.color':'w', 'text.color':'w', 'axes.labelcolor':'w'})



In [2]:
!pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [3]:
os.environ['KAGGLE_USERNAME'] = "cristinavillagran"
os.environ['KAGGLE_KEY'] = "72746c01e1eea1023b86a11154fc77b3"

#https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip -o 'imdb-dataset-of-50k-movie-reviews.zip'

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [5]:
os.listdir()

['.config',
 '7282_1.csv',
 'hotel-reviews.zip',
 'imdb-dataset-of-50k-movie-reviews.zip',
 'winemag-data_first150k.csv',
 'winemag-data-130k-v2.json',
 'Datafiniti_Hotel_Reviews.csv',
 'Datafiniti_Hotel_Reviews_Jun19.csv',
 'wine-reviews.zip',
 'IMDB Dataset.csv',
 'winemag-data-130k-v2.csv',
 'sample_data']

In [6]:
movie = pd.read_csv('IMDB Dataset.csv')
movie.head

<bound method NDFrame.head of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>

In [7]:
movie.columns

Index(['review', 'sentiment'], dtype='object')

In [8]:
movie.review

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [9]:
movie.sentiment

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object

In [0]:
movie['es_positivo']=movie['sentiment'].apply(lambda x: 1 if x == "positive" else 0)


In [12]:
movie.drop(columns=['sentiment'])

,review,es_positivo
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [0]:
from sklearn.model_selection import train_test_split
review = movie.review
positivo = movie.es_positivo

review_train,positivo_test,positivo_train,review_test = train_test_split(review,positivo,test_size=0.30)

In [0]:
max_words = 1000
max_len = 150
epochs = 5
STEPS_PER_EPOCH = 32
VALIDATION_STEPS = 16
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(review)
sequences = tok.texts_to_sequences(review)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [20]:
model = Sequential([
    Embedding(max_words, 32, mask_zero=True),
    Bidirectional(LSTM(64)),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          32000     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               49664     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 81,793
Trainable params: 81,793
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
model.fit(sequences_matrix,positivo,batch_size=128,epochs=3,
          validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/3
40000/40000 [==============================] - 198s 5ms/sample - loss: 0.3277 - accuracy: 0.8601 - val_loss: 0.3337 - val_accuracy: 0.8545
Epoch 2/3
40000/40000 [==============================] - 198s 5ms/sample - loss: 0.3144 - accuracy: 0.8661 - val_loss: 0.3225 - val_accuracy: 0.8568
Epoch 3/3
40000/40000 [==============================] - 199s 5ms/sample - loss: 0.3050 - accuracy: 0.8700 - val_loss: 0.3306 - val_accuracy: 0.8594


In [0]:
test_sequences = tok.texts_to_sequences(review)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [37]:
accr = model.evaluate(test_sequences_matrix,positivo)
print('Loss: {:0.3f} y  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

50000/50000 [==============================] - 71s 1ms/sample - loss: 0.2934 - accuracy: 0.8763
Loss: 0.293 y  Accuracy: 0.876
